# Notebook Management Utility

This notebook provides utilities for managing Jupyter notebook files in a development workflow. It helps maintain separate versions of notebooks for version control and local development.

## Key Features
- Creates `*_local.ipynb` copies of notebooks for local development
- Strips output cells from original notebooks for clean version control
- Dry-run option to preview changes before execution
- Interactive confirmation before applying changes
- Safe execution with preview and confirmation workflow

## Usage Example
```python
# Preview changes (dry run)
prep_notebooks(Path("./notebooks"), dry_run=True)

# Execute changes with confirmation (see last cell block)
from tnh_scholar.utils import get_user_confirmation

dir_to_mod = Path("./notebooks")
prep_notebooks(dir_to_mod, dry_run=True)  # Preview changes

if get_user_confirmation("Proceed with changes? (y/N)", default=False):
    prep_notebooks(dir_to_mod, dry_run=False)
```

The utility implements a safe workflow for notebook management:
1. First shows a preview of planned changes (dry run)
2. Prompts for user confirmation
3. Only executes changes after explicit confirmation

This helps prevent accidental modifications while maintaining clean notebooks in version control with separate local development copies.

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from tnh_scholar.utils import get_user_confirmation

In [39]:
import sys
from pathlib import Path
import shutil
import subprocess

def prep_notebooks(directory: Path, dry_run: bool = True) -> bool:
    """
    Prepare notebooks in directory:
    1. Copy *.ipynb to *_local.ipynb if doesn't exist
    2. Strip outputs from original notebooks using nbconvert
    """
    directory = Path(directory)
    if not directory.exists():
        print(f"Directory not found: {directory}")
        return False

    # Find all notebooks that don't end in _local.ipynb
    notebooks = list(directory.rglob("*.ipynb"))
    notebooks = [path for path in notebooks if ".ipynb_checkpoints" not in str(path)]
    
    #nb_list = [str(nb) for nb in filtered_notebooks] 
    #print("All notebooks found:")
    #print("\n".join(nb_list))

    print(f"Found {len(notebooks)} notebooks to process in {directory}. Ignoring all checkpoint notebooks.")

    for nb_path in notebooks:
        local_path = nb_path.parent / f"{nb_path.stem}_local{nb_path.suffix}"

        # Copy to local version if doesn't exist
        if local_path.exists():
            print(f"local copy of notebook exists: {local_path}")
        elif dry_run:
            print(f"Would copy: {nb_path} -> {local_path}")
        else:
            print(f"Copying: {nb_path} -> {local_path}")
            shutil.copy2(nb_path, local_path)

        # Strip outputs from original using nbconvert
        if dry_run:
            print(f"Would strip outputs from: {nb_path}")
        else:
            print(f"Stripping outputs from: {nb_path}")
            subprocess.run([
                "jupyter", "nbconvert", 
                "--ClearOutputPreprocessor.enabled=True",
                "--inplace",
                str(nb_path)
            ])
    
    return True


In [40]:
dir_to_mod = Path("./notebooks")  # set path to "./notebooks" to run on local notebooks dir.

if not dir_to_mod.exists():
    print("Directory not found. Aborted.")
else:
    print("Running dry run first...")

    if prep_notebooks(dir_to_mod, dry_run=True):

        if proceed := get_user_confirmation(
            "\nProceed with these changes? (y/N)", default=False
        ):
            print("\nExecuting changes...")
            prep_notebooks(dir_to_mod, dry_run=False)
        else:
            print("Aborted.")

Running dry run first...
Found 71 notebooks to process in notebooks. Ignoring all checkpoint notebooks.
Would copy: notebooks/video_processing/postprocessing_english.ipynb -> notebooks/video_processing/postprocessing_english_local.ipynb
Would strip outputs from: notebooks/video_processing/postprocessing_english.ipynb
Would copy: notebooks/video_processing/process_stepwise.ipynb -> notebooks/video_processing/process_stepwise_local.ipynb
Would strip outputs from: notebooks/video_processing/process_stepwise.ipynb
Would copy: notebooks/video_processing/repair_process.ipynb -> notebooks/video_processing/repair_process_local.ipynb
Would strip outputs from: notebooks/video_processing/repair_process.ipynb
Would copy: notebooks/video_processing/collating.ipynb -> notebooks/video_processing/collating_local.ipynb
Would strip outputs from: notebooks/video_processing/collating.ipynb
Would copy: notebooks/video_processing/youtube_playlist_process.ipynb -> notebooks/video_processing/youtube_playlist_

y

Executing changes...
Found 71 notebooks to process in notebooks. Ignoring all checkpoint notebooks.
Copying: notebooks/video_processing/postprocessing_english.ipynb -> notebooks/video_processing/postprocessing_english_local.ipynb
Stripping outputs from: notebooks/video_processing/postprocessing_english.ipynb


[NbConvertApp] Converting notebook notebooks/video_processing/postprocessing_english.ipynb to notebook
[NbConvertApp] Writing 21225 bytes to notebooks/video_processing/postprocessing_english.ipynb


Copying: notebooks/video_processing/process_stepwise.ipynb -> notebooks/video_processing/process_stepwise_local.ipynb
Stripping outputs from: notebooks/video_processing/process_stepwise.ipynb


[NbConvertApp] Converting notebook notebooks/video_processing/process_stepwise.ipynb to notebook
[NbConvertApp] Writing 6485 bytes to notebooks/video_processing/process_stepwise.ipynb


...

[NbConvertApp] Converting notebook notebooks/video_processing/tests_and_prototypes/audio_extract_testing4.ipynb to notebook
[NbConvertApp] Writing 8265 bytes to notebooks/video_processing/tests_and_prototypes/audio_extract_testing4.ipynb


Copying: notebooks/video_processing/tests_and_prototypes/audio_extract_testing2.ipynb -> notebooks/video_processing/tests_and_prototypes/audio_extract_testing2_local.ipynb
Stripping outputs from: notebooks/video_processing/tests_and_prototypes/audio_extract_testing2.ipynb


[NbConvertApp] Converting notebook notebooks/video_processing/tests_and_prototypes/audio_extract_testing2.ipynb to notebook
[NbConvertApp] Writing 5442 bytes to notebooks/video_processing/tests_and_prototypes/audio_extract_testing2.ipynb


Copying: notebooks/video_processing/tests_and_prototypes/post_processing_tests.ipynb -> notebooks/video_processing/tests_and_prototypes/post_processing_tests_local.ipynb
Stripping outputs from: notebooks/video_processing/tests_and_prototypes/post_processing_tests.ipynb


[NbConvertApp] Converting notebook notebooks/video_processing/tests_and_prototypes/post_processing_tests.ipynb to notebook
[NbConvertApp] Writing 10017 bytes to notebooks/video_processing/tests_and_prototypes/post_processing_tests.ipynb


In [30]:
".ipynb_checkpoints" in str(Path("notebooks/testing/.ipynb_checkpoints/yt_fetch_test-checkpoint.ipynb"))

True

In [29]:
str(Path("notebooks/testing/.ipynb_checkpoints/yt_fetch_test-checkpoint.ipynb"))

'notebooks/testing/.ipynb_checkpoints/yt_fetch_test-checkpoint.ipynb'